In [8]:
import mysql.connector as mysqlc
import pandas as pd
from pathlib import Path
import os

In [9]:
home_dir = os.path.expanduser('~')
os.chdir(home_dir)

In [10]:
host = '127.0.0.1'
user = 'root' 
password = 'ethan1' 
database = 'ai_proj_2025'

try: 
    conn = mysqlc.connect(
        host = host,
        user = user, 
        password = password, 
        database = database
    )
    print("success")
    # conn.close()
except mysqlc.Error as err:
    print(f"Error {err}")

success


The following two paths will be used to access information that will be placed into the database. 

In [11]:
data_path = Path(home_dir, 'OneDrive - Stephen F. Austin State University', 'CrisisMMD_v2.0','CrisisMMD_v2.0')
annot_path = data_path / 'annotations'

print(annot_path)

C:\Users\bowdenaa\OneDrive - Stephen F. Austin State University\CrisisMMD_v2.0\CrisisMMD_v2.0\annotations


In [12]:
print(data_path)
data_path.exists()

C:\Users\bowdenaa\OneDrive - Stephen F. Austin State University\CrisisMMD_v2.0\CrisisMMD_v2.0


True

In [13]:

harvey_annot = pd.read_csv(annot_path / 'hurricane_harvey_final_data.tsv', sep='\t')


In [14]:
url_max_len = harvey_annot['image_url'].apply(len).max()
url_max_len # just seeing how many character the column in sql should be

84

In [15]:
harvey_annot.head()

,tweet_id,image_id,text_info,text_info_conf,image_info,image_info_conf,text_human,text_human_conf,image_human,image_human_conf,image_damage,image_damage_conf,tweet_text,image_url,image_path
0,905274232590004225,901671684478029825_0,not_informative,1.0000,informative,1.0,not_humanitarian,1.0000,infrastructure_and_utility_damage,1.0,severe_damage,1.0000,"CONGRATS ON HITTING YOIR GOAL GUYS, I'm sure t...",http://pbs.twimg.com/media/DINhGs8XcAE1H7q.jpg,data_image/hurricane_harvey/27_8_2017/90167168...
1,901646074527535105,901646074527535105_0,informative,0.6822,informative,1.0,injured_or_dead_people,0.6822,infrastructure_and_utility_damage,1.0,severe_damage,0.6728,RT @ajwamood: #ajwamood : Harvey the first maj...,http://pbs.twimg.com/media/DILxh_xWAAAfJDY.jpg,data_image/hurricane_harvey/27_8_2017/90164607...
2,901646074527535105,901646074527535105_1,informative,0.6822,informative,1.0,injured_or_dead_people,0.6822,infrastructure_and_utility_damage,1.0,severe_damage,1.0000,RT @ajwamood: #ajwamood : Harvey the first maj...,http://pbs.twimg.com/media/DILxiisXYAAokz_.jpg,data_image/hurricane_harvey/27_8_2017/90164607...
3,901646074527535105,901646074527535105_2,informative,0.6822,informative,1.0,injured_or_dead_people,0.6822,infrastructure_and_utility_damage,1.0,severe_damage,0.6528,RT @ajwamood: #ajwamood : Harvey the first maj...,http://pbs.twimg.com/media/DILxjSUWAAEIaWH.jpg,data_image/hurricane_harvey/27_8_2017/90164607...
4,901646074527535105,901646074527535105_3,informative,0.6822,informative,1.0,injured_or_dead_people,0.6822,infrastructure_and_utility_damage,1.0,severe_damage,0.6812,RT @ajwamood: #ajwamood : Harvey the first maj...,http://pbs.twimg.com/media/DILxkWJXgAANF-E.jpg,data_image/hurricane_harvey/27_8_2017/90164607...


In [16]:
create_text_table_query = '''
CREATE Table Tweets(
    tweet_id VARCHAR(50) PRIMARY KEY, 
    event VARCHAR(30), 
    text_info VARCHAR(50) NOT NULL,
    text_info_conf DECIMAL(5,4),
    text_human VARCHAR(50), 
    text_human_conf DECIMAL(5,4), 
    tweet_text VARCHAR(280)
);
'''

In [17]:
create_image_table_query = '''
CREATE TABLE Images (
    idx int NOT NULL AUTO_INCREMENT PRIMARY KEY,
    image_id VARCHAR(50) NOT NULL, 
    tweet_id VARCHAR(50),
    FOREIGN KEY (tweet_id) REFERENCES Tweets(tweet_id),
    image_path VARCHAR(200) NOT NULL, 
    image_info VARCHAR(50) NOT NULL, 
    image_info_conf DECIMAL(5,4), 
    image_human VARCHAR(50),
    image_human_conf DECIMAL(5,4), 
    image_damage VARCHAR(50),
    image_damage_conf DECIMAL(5,4), 
    image_url VARCHAR(100),
    date DATETIME
);
'''

In [18]:
cursor = conn.cursor()

## Inserting into the database
1. Access the annotations file
2. Insert the information from the annotations folder into the database
3. To get the event, we have to track what folder we are in because the names of the folders correspond to the event

In [19]:
annotations_per_event = os.listdir(annot_path) # read all the file names in the folder
annotations_per_event = [file_str for file_str in annotations_per_event if not file_str.startswith('._') and 'hurricane' in file_str] # the ._ files don't work for Windows OS
annotations_per_event

['hurricane_harvey_final_data.tsv',
 'hurricane_irma_final_data.tsv',
 'hurricane_maria_final_data.tsv']

In [20]:
image_dir_path = data_path / 'data_image' 

In [21]:
image_folders = os.listdir(image_dir_path)
image_folders = [folder for folder in image_folders if 'hurricane' in folder]
image_folders

['hurricane_harvey', 'hurricane_irma', 'hurricane_maria']

Because the images are organized into folders event_name => day_related_to_disaster we will need to ensure that our SQL can parse the dates so we can insert them into our database as column values. 

In [22]:
# create a fn that can convert the folder name with the date into a string that sql can recognize
def folder_to_date(folder_name):
    return f"STR_TO_DATE('{folder_name}', '%d_%m_%Y')"

In [23]:
# testing the function out
cursor.execute(f'SELECT {folder_to_date('19_9_2017')}')
cursor.fetchall()

[(datetime.date(2017, 9, 19),)]

### Populating the Database

Now, we must create a function that can insert the data into our database. Let's break down what we will be inserting like so:

1. From the annotations folder

   This is the step where we access all information except for getting the actual image itself, which we will access in the next step using the image_path attribute we get from this step. 
   1. By event
3. Getting the images
   1. By event
   2. By date 

In [25]:
import re
def populate_db():
    for f in annotations_per_event: # annotations_per_event is the list of .tsv files we created earlier
        path = annot_path / f
        df = pd.read_csv(path, sep='\t') # a tsv file in the annotations folder for a specific event
        df = df.where(pd.notnull(df), None)

        for (_, row) in df.iterrows(): # i is the idx, j is the series
            
            img_path = row.loc['image_path'] # will also use this to get the event name
            date_pattern = r'(\d{1,2}_\d{1,2}_\d{4})' # day month year - naming format of folders that the images are in 
            date = re.findall(date_pattern, img_path)[0]
            
            # # PART 2 - Inserting into the Tweets table 
            # event_pattern = r'data_image\/(\w+)\/\d{1,2}_\d{1,2}_\d{4}' 
            # event = re.findall(event_pattern, img_path)[0]
            
            # sql_text1 = """
            # INSERT INTO Tweets (tweet_id, event, text_info, text_info_conf, text_human, text_human_conf, tweet_text) 
            # VALUES (%s, %s, %s, %s, %s, %s, %s)
            # """
            # cursor.execute(f"SELECT * FROM Tweets WHERE tweet_id='{row.loc['tweet_id']}'")
            # if not cursor.fetchall():
            #     cursor.execute(sql_text1, (
            #         row['tweet_id'], event, row['text_info'], row['text_info_conf'], row['text_human'],
            #         row['text_human_conf'], row['tweet_text']
            #     ))
            
            # PART 1 - Inserting into the Images table
            
            cursor.execute(f"SELECT * FROM Hurricane_Images WHERE image_id='{row.loc['image_id']}'")
            if not cursor.fetchall():
                sql_img1 = sql_img1 = f"""
                    INSERT INTO Hurricane_Images (image_id, tweet_id, image_path, image_info, image_info_conf, image_human, image_human_conf, image_damage, image_damage_conf, image_url, date) 
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, {folder_to_date(date)})
                    """
                cursor.execute(sql_img1, (
                    row['image_id'], row['tweet_id'], row['image_path'], row['image_info'], row['image_info_conf'],
                    row['image_human'], row['image_human_conf'], row['image_damage'], row['image_damage_conf'],
                    row['image_url']
                ))
            

                
        

In [26]:
populate_db()
conn.commit()
cursor.close()
conn.close()

In [28]:
'''
CREATE Table Tweets(
    tweet_id int PRIMARY KEY, 
    event VARCHAR(30), 
    text_info VARCHAR(50) NOT NULL,
    text_info_conf DECIMAL(5,4),
    text_human VARCHAR(50), 
    text_human_conf DECIMAL(5,4), 
    tweet_text VARCHAR(280)
);
'''

'\nCREATE Table Tweets(\n    tweet_id int PRIMARY KEY, \n    event VARCHAR(30), \n    text_info VARCHAR(50) NOT NULL,\n    text_info_conf DECIMAL(5,4),\n    text_human VARCHAR(50), \n    text_human_conf DECIMAL(5,4), \n    tweet_text VARCHAR(280)\n);\n'

Make sure to close the connection once you're finished! 